![Img](https://app.theheadstarter.com/static/hs-logo-opengraph.png)

# Headstarter Codebase RAG Project

![Screenshot 2024-11-25 at 7 12 58 PM](https://github.com/user-attachments/assets/48dd9de1-b4d2-4318-8f52-85ec209d8ebc)

# Install Necessary Libraries

In [4]:
! pip install pygithub langchain langchain-community openai tiktoken pinecone-client langchain_pinecone sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 830.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5

In [6]:
from google.colab import userdata

In [5]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata
from pinecone import Pinecone
import os
import tempfile
from github import Github, Repository
from git import Repo
from openai import OpenAI
from pathlib import Path
from langchain.schema import Document
from pinecone import Pinecone
from groq import Groq

# Clone a GitHub Repo locally

In [5]:
def clone_repository(repo_url):
    """Clones a GitHub repository to a temporary directory.

    Args:
        repo_url: The URL of the GitHub repository.

    Returns:
        The path to the cloned repository.
    """

    # github_repo = "https://github.com/CoderAgent/SecureAgent"
    repo_name = repo_url.split("/")[-1]
    repo_path = f"/content/{repo_name}"
    Repo.clone_from(repo_url, str(repo_path))
    return repo_path

In [16]:
path = "https://github.com/CoderAgent/SecureAgent"

# Define which types of files to parse and which files / folders to ignore

In [15]:
SUPPORTED_EXTENSIONS = {'.py', '.js', '.tsx', '.jsx', '.ipynb', '.java',
                         '.cpp', '.ts', '.go', '.rs', '.vue', '.swift', '.c', '.h'}

IGNORED_DIRS = {'node_modules', 'venv', 'env', 'dist', 'build', '.git',
                '__pycache__', '.next', '.vscode', 'vendor'}

In [13]:
def get_file_content(file_path, repo_path):
    """
    Get content of a single file.

    Args:
        file_path (str): Path to the file

    Returns:
        Optional[Dict[str, str]]: Dictionary with file name and content
    """

    try:
        with open(file_path, "r", encoding="utf-8") as f:
          content = f.read()

        rel_path = os.path.relpath(file_path, repo_path)
        return {
            "name": rel_path,
            "content": content
        }
    except Exception as e:
        print(f"Error processing file {file_path}: {str(e)}")
        return None


def get_main_files_content(repo_path: str):
    """
    Get content of supported code files from the local repository.

    Args:
        repo_path: Path to the local repository

    Returns:
        List of dictionaries containing file names and contents
    """

    files_content = []

    try:
        for root, _, files in os.walk(repo_path): # _ means unused variable
            if any(ignored_dir in root for ignored_dir in IGNORED_DIRS):
                continue
            #process the rest of the files
            for file in files:
                file_path = os.path.join(root, file)
                # check if the current file has a supported extention
                if os.path.splitext(file)[1] in SUPPORTED_EXTENSIONS: # checks if the file extention is the in the set
                    file_content = get_file_content(file_path, repo_path)

                    if file_content:
                        files_content.append(file_content)

        #return files_content
    except Exception as e:
        print(e)

    return files_content


In [17]:
files_content = get_main_files_content(path)

In [ ]:
files_content

# Embeddings

In [18]:
def get_huggingface_embeddings(text, model_name="sentence-transformers/all-mpnet-base-v2"):
    model = SentenceTransformer(model_name)
    return model.encode(text)

  # free and relatively fast hugging_face

In [18]:
text = "I like coding"

embeddings = get_huggingface_embeddings(text)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [22]:
embeddings # basically a 768 dimentional vector that captures the information in the text
# once converted into numbers, can see how similar they are using cosine similarity
# or osme other kind of linalg

array([-8.99214391e-03,  7.10570812e-02, -4.78132404e-02,  2.93951924e-03,
        1.87183879e-02,  6.25578165e-02, -7.40241334e-02, -1.79869793e-02,
       -2.33030953e-02,  2.52969563e-02,  1.87584516e-02,  3.26438658e-02,
        2.42187642e-02,  1.06377289e-01,  1.96239557e-02,  4.58664587e-03,
       -2.44689845e-02, -2.76926570e-02, -3.92995449e-03, -1.75862219e-02,
        7.31050968e-03,  3.00472043e-02, -3.48030999e-02, -1.51053397e-02,
        4.95744497e-03, -1.16299819e-02,  2.20015477e-02, -6.58035874e-02,
       -2.77131312e-02,  1.00825511e-01,  3.28472641e-04, -9.48085822e-03,
       -2.19271723e-02, -4.70387796e-03,  1.50509811e-06,  1.31359547e-02,
       -1.65854990e-02,  1.93589348e-02, -2.62506306e-02,  1.61743432e-04,
       -1.85249373e-02, -1.79329310e-02, -2.03854069e-02,  2.96985377e-02,
       -5.77551797e-02,  1.27474498e-02,  8.77798274e-02, -6.04865141e-02,
        9.92059615e-03, -2.96356127e-04,  1.08571432e-03, -1.75144020e-02,
       -3.35006081e-02, -

# Setting up Pinecone
**1. Create an account on [Pinecone.io](https://app.pinecone.io/)**

**2. Create a new index called "codebase-rag" and set the dimensions to 768. Leave the rest of the settings as they are.**

![Screenshot 2024-11-24 at 10 58 50 PM](https://github.com/user-attachments/assets/f5fda046-4087-432a-a8c2-86e061005238)



**3. Create an API Key for Pinecone**

![Screenshot 2024-11-24 at 10 44 37 PM](https://github.com/user-attachments/assets/e7feacc6-2bd1-472a-82e5-659f65624a88)


**4. Store your Pinecone API Key within Google Colab's secrets section, and then enable access to it (see the blue checkmark)**

![Screenshot 2024-11-24 at 10 45 25 PM](https://github.com/user-attachments/assets/eaf73083-0b5f-4d17-9e0c-eab84f91b0bc)



In [19]:
# Set the PINECONE_API_KEY as an environment variable
pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key

# Initialize Pinecone
pc = Pinecone(api_key=userdata.get("PINECONE_API_KEY"),)

# Connect to your Pinecone index
pinecone_index = pc.Index("codebase-rag")

In [20]:
vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())

<ipython-input-20-5982ffb8f713>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())
<ipython-input-20-5982ffb8f713>:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [21]:
# Insert the codebase embeddings into Pinecone
documents = []
# Document data structure imported from langchain
# 2 properties - page content and metadat
for file in files_content:
    doc = Document(
        page_content = f"{file['name']}\n\n{file['content']}",
        metadata={"source": file['name']}
    )

    documents.append(doc)

vectorstore = PineconeVectorStore.from_documents(
    documents=documents,
    embedding=HuggingFaceEmbeddings(),
    index_name="codebase-rag",
    namespace="https://github.com/CoderAgent/SecureAgent"
)

<ipython-input-21-79b5aa74436c>:15: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding=HuggingFaceEmbeddings(),


In [26]:
documents

[Document(metadata={'source': 'src/review-agent.ts'}, page_content='src/review-agent.ts\n\nimport { Octokit } from "@octokit/rest";\nimport { WebhookEventMap } from "@octokit/webhooks-definitions/schema";\nimport { ChatCompletionMessageParam } from "groq-sdk/resources/chat/completions";\nimport * as xml2js from "xml2js";\nimport type {\n  BranchDetails,\n  BuilderResponse,\n  Builders,\n  CodeSuggestion,\n  PRFile,\n  PRSuggestion,\n} from "./constants";\nimport { PRSuggestionImpl } from "./data/PRSuggestionImpl";\nimport { generateChatCompletion } from "./llms/chat";\nimport {\n  PR_SUGGESTION_TEMPLATE,\n  buildPatchPrompt,\n  constructPrompt,\n  getReviewPrompt,\n  getTokenLength,\n  getXMLReviewPrompt,\n  isConversationWithinLimit,\n} from "./prompts";\nimport {\n  INLINE_FIX_FUNCTION,\n  getInlineFixPrompt,\n} from "./prompts/inline-prompt";\nimport { getGitFile } from "./reviews";\n\nexport const reviewDiff = async (messages: ChatCompletionMessageParam[]) => {\n  const message = a

# Perform RAG

1. Get your OpenRouter API Key [here](https://openrouter.ai/settings/keys)

2. Paste your OpenRouter Key into your Google Colab secrets, and make sure to enable permissions for it

![Image](https://github.com/user-attachments/assets/bd64c5aa-952e-4a1e-9ac0-01d8fe93aaa1)


In [22]:
#using openai to access openrouter
#lot of different api providers use the same api setup as openai
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=userdata.get("OPENROUTER_API_KEY")
)

In [45]:
query = "How is the javascript parser used?"
#using the embeddingfs in pinecone -> find the most relevant files for the question

In [46]:
# get the embedding for the query
raw_query_embedding = get_huggingface_embeddings(query)

In [ ]:
raw_query_embedding

In [47]:
top_matches = pinecone_index.query(vector=raw_query_embedding.tolist(), top_k=5,
                                   include_metadata=True,
                                   namespace="https://github.com/CoderAgent/SecureAgent")

In [ ]:
top_matches

In [48]:
#extract the right information from the data structure
context = [item['metadata']['text'] for item in top_matches['matches']]

In [ ]:
context

In [24]:
augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(context) + "\n-------\n<\CONTEXT>\n\n\n\nMY QUESTION:\n" + query

NameError: name 'context' is not defined

In [ ]:
print(augmented_query)

In [23]:
# thats it to call a deepseek api
system_prompt = """You are a Senior Software Engineer, who is an expert in
                TypeScript. Answer the questions I have about the codebase based
                on the context provides. Always consider all of the context provided
                to answer my question.
                """

llm_response = client.chat.completions.create(
    model = "deepseek/deepseek-r1:free",
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": augmented_query}
    ]
)

response = llm_response.choices[0].message.content

NameError: name 'augmented_query' is not defined

In [51]:
print(response)

The JavaScript parser is used in two key ways within this codebase:

1. **As part of file validation**:
```typescript
// Through the dryRun method
const parser = new JavascriptParser();
const result = parser.dryRun(code);
// Returns whether JS/TS code is syntactically valid
```

2. **For context-aware code review**:
```typescript
// When processing JS/TS files in code review diffs
const parser = getParserForExtension("src/components/Button.tsx"); 
// Returns JavascriptParser instance

// The parser is then used to:
parser.findEnclosingContext(fileContents, startLine, endLine);
// Finds the largest enclosing context (function/interface) containing the changed lines
```

Key implementation details:
- Processes JavaScript and TypeScript files via Babel parser
- Looks for both function declarations and TypeScript interfaces
- Tracks the largest enclosing context to determine code scope for diffs
- Integrated into the review system via `smarterContextPatchStrategy`
- Filters AST nodes using

In [25]:
def perform_rag(query):
    raw_query_embedding = get_huggingface_embeddings(query)

    top_matches = pinecone_index.query(vector=raw_query_embedding.tolist(), top_k=5,
                                   include_metadata=True,
                                   namespace="https://github.com/CoderAgent/SecureAgent")
    context = [item['metadata']['text'] for item in top_matches['matches']]

    augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(context) + "\n-------\n<\CONTEXT>\n\n\n\nMY QUESTION:\n" + query


    system_prompt = """You are a Senior Software Engineer, who is an expert in
                TypeScript. Answer the questions I have about the codebase based
                on the context provides. Always consider all of the context provided
                to answer my question.
                """

    llm_response = client.chat.completions.create(
        model = "deepseek/deepseek-r1:free",
        messages = [
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": augmented_query}
        ]
    )

    response = llm_response.choices[0].message.content
    return response


In [1]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 4.2 MB/s eta 0:00:00


In [7]:


groq_client = Groq(
    api_key=userdata.get("GROQ_API_KEY")
)

In [8]:
def get_poem(query):
    groq_prompt = """You are an expert on poetry and love to make poems about code.
                  Given a description about a codebase, rewrite the description in
                  the form of a series of haikus.

                  """

    completions = groq_client.chat.completions.create(
        model = "llama-3.3-70b-versatile",
        messages = [
            {
                "role": "system",
                "content": groq_prompt
            },
            {
                "role": "user",
                "content": query
            }
        ]
    )

    response = completions.choices[0].message.content
    return response

Reinitialized existing Git repository in /content/.git/
Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@90fb59193599.(none)')
error: remote origin already exists.
error: src refspec main does not match any
error: failed to push some refs to 'https://github.com/Mitul-Marimuthu/RAG-EXAMPLE.git'


In [26]:
openai_completion = perform_rag("How many parsers are there? list them")
groq_completion = get_poem(openai_completion)

print(groq_completion)

Parsers two abide
Python and JavaScript reign
Codebase's gentle

Python parser born
Basic interface in place
Placeholder's calm sleep

Javascript's might
Babel parser's strong backbone
AST traversal

Context aware soul
Enclosing functions sought
Logic's subtle dance

Abstract bond ties
Interface implemented by
Both parsers as one

Review system's eye
Analyzes code changes deep
Pull requests scrutin

JavaScript's pride
ECMAScript's complex
Features handled with

Sophistication
Shines in JavaScript parser
Codebase's delight
